# Surprise Housing Assignment
## Author: Kumaraguru Muthuraj
### Problem Statement
#### BoomBikes wants to understand the meteorological and human lifestyle patterns that affect the shared bike demand in USA.


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model, metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import os

# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [11]:
housingDF = pd.read_csv("train.csv")

print(housingDF.shape)
housingDF.head()


(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
housingDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [12]:
housingDF['MSZoning'].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

In [41]:
catCols = housingDF.select_dtypes(include=['object'])
cols = catCols.columns

for col in cols:
    total = catCols[col].size
    notNull = catCols[col].count()
    if  notNull != total:
        print('**********************')
        print(col, '**** Total count ',  total, '**** Not null ', notNull)
        print(catCols[col].value_counts())
        print(' ')


**********************
Alley **** Total count  1460 **** Not null  91
Grvl    50
Pave    41
Name: Alley, dtype: int64
 
**********************
MasVnrType **** Total count  1460 **** Not null  1452
None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64
 
**********************
BsmtQual **** Total count  1460 **** Not null  1423
TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64
 
**********************
BsmtCond **** Total count  1460 **** Not null  1423
TA    1311
Gd      65
Fa      45
Po       2
Name: BsmtCond, dtype: int64
 
**********************
BsmtExposure **** Total count  1460 **** Not null  1422
No    953
Av    221
Gd    134
Mn    114
Name: BsmtExposure, dtype: int64
 
**********************
BsmtFinType1 **** Total count  1460 **** Not null  1423
Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
Name: BsmtFinType1, dtype: int64
 
**********************
BsmtFinType2 **** Total count  1460 **** Not null  1422
U